In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data = data.loc[data['Year']== '2016']
data

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
14904,17069,This Thing Called Life,August Alsina,ever since august alsina emerged in 2012 with ...,Def Jam,7.4,January,5,2016,Pop/R&B,Rap,NaN,NaN
14908,17074,Darkest Before Dawn,Pusha-T,pusha t occupies a unique spot in hip-hop. he'...,"Def Jam,G.O.O.D. Music",8.2,January,4,2016,Rap,NaN,NaN,NaN
14909,17075,Emotional Mugger,Ty Segall,whether under his own name or from various sid...,Drag City,7.3,January,19,2016,Rock,NaN,NaN,NaN
14910,17076,Screamadelica,Primal Scream,"a trumpet riff; a louche, echoey, funk groove ...","Creation,Creation",9.0,January,4,2016,Rock,NaN,NaN,NaN
14911,17077,Another Side,Fingers Inc.,"""i used to pass the warehouse after school, on...",Alleviated,8.8,January,12,2016,Electronic,NaN,NaN,NaN
14913,17079,The Catastrophist,Tortoise,"tortoise, the chicago-based instrumental post-...",Thrill Jockey,6.8,January,19,2016,Experimental,Rock,NaN,NaN
14914,17080,Film Music,Mikael Tariverdiev,arrives in the west with little in the way of...,Earth,8.0,January,7,2016,Global,NaN,NaN,NaN
14915,17081,HITNRUN Phase Two,Prince,"as a thought experiment, it's fun to imagine h...",NPG,4.7,January,8,2016,Pop/R&B,Rock,NaN,NaN
14916,17082,AGE 101: DROP X,Little Simz,few young rappers are as explicitly vulnerable...,Self-released,7.4,January,21,2016,Rap,NaN,NaN,NaN
14917,17083,Northern Lights,Allan Kingdom,"last february, , brought 40 , men (and at leas...",Self-released,7.5,January,19,2016,Rap,NaN,NaN,NaN


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['content']))

print(data_words[:1])

[['ever', 'since', 'august', 'alsina', 'emerged', 'in', 'with', 'his', 'first', 'official', 'mixtape', 'he', 'been', 'somewhat', 'of', 'curio', 'in', 'essence', 'he', 'rapper', 'in', 'an', 'singer', 'body', 'more', 'likely', 'to', 'sing', 'about', 'his', 'harrowed', 'past', 'and', 'the', 'things', 'he', 'done', 'to', 'keep', 'food', 'on', 'the', 'table', 'for', 'his', 'family', 'than', 'some', 'romantic', 'tryst', 'even', 'would', 'you', 'know', 'the', 'lone', 'love', 'song', 'on', 'the', 'new', 'orleans', 'native', 'latest', 'album', 'is', 'conflicted', 'ballad', 'on', 'which', 'alsina', 'is', 'ball', 'of', 'nerves', 'afraid', 'that', 'opening', 'up', 'to', 'the', 'possibility', 'of', 'love', 'would', 'leave', 'him', 'vulnerable', 'but', 'alsina', 'applies', 'the', 'tender', 'ruminative', 'aspects', 'of', 'and', 'its', 'sense', 'of', 'longing', 'especially', 'to', 'his', 'street', 'tales', 'and', 'his', 'second', 'studio', 'album', 'solidifies', 'his', 'place', 'amongst', 'the', 'uppe

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['ever', 'since', 'august', 'alsina', 'emerged', 'in', 'with', 'his', 'first', 'official', 'mixtape', 'he', 'been', 'somewhat', 'of', 'curio', 'in', 'essence', 'he', 'rapper', 'in', 'an', 'singer', 'body', 'more', 'likely', 'to', 'sing', 'about', 'his', 'harrowed', 'past', 'and', 'the', 'things', 'he', 'done', 'to', 'keep', 'food', 'on', 'the', 'table', 'for', 'his', 'family', 'than', 'some', 'romantic', 'tryst', 'even', 'would', 'you', 'know', 'the', 'lone', 'love', 'song', 'on', 'the', 'new_orleans', 'native', 'latest', 'album', 'is', 'conflicted', 'ballad', 'on', 'which', 'alsina', 'is', 'ball', 'of', 'nerves', 'afraid', 'that', 'opening', 'up', 'to', 'the', 'possibility', 'of', 'love', 'would', 'leave', 'him', 'vulnerable', 'but', 'alsina', 'applies', 'the', 'tender', 'ruminative', 'aspects', 'of', 'and', 'its', 'sense', 'of', 'longing', 'especially', 'to', 'his', 'street', 'tales', 'and', 'his', 'second', 'studio', 'album', 'solidifies', 'his', 'place', 'amongst', 'the', 'upper', 

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['ever', 'august', 'alsina', 'emerge', 'first', 'official', 'mixtape', 'somewhat', 'curio', 'essence', 'rapper', 'singer', 'body', 'likely', 'sing', 'harrow', 'past', 'thing', 'do', 'keep', 'food', 'table', 'family', 'romantic', 'tryst', 'even', 'would', 'know', 'lone', 'love', 'song', 'new_orlean', 'native', 'late', 'album', 'conflicted', 'ballad', 'alsina', 'ball', 'nerve', 'afraid', 'open', 'possibility', 'love', 'would', 'leave', 'vulnerable', 'alsina', 'apply', 'tender', 'ruminative', 'aspect', 'sense', 'long', 'especially', 'street', 'tale', 'second', 'studio', 'album', 'solidify', 'place', 'upper', 'echelon', 'modern', 'day', 'many', 'contemporary', 'alsina', 'share', 'interest', 'even', 'explore', 'percussive', 'element', 'rap', 'myriad', 'way', 'marry', 'melody', 'sepia', 'tone', 'lead_single', 'effortlessly', 'slip', 'back', 'forth', 'sweet', 'rhythmic', 'singing', 'melody', 'orient', 'rapping', 'describe', 'environment', 'grow', 'come', 'criminal', 'way', 'chorus', 'dreamer

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 3), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 2), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"session" + 0.008*"performance" + 0.008*"material" + '
  '0.008*"recording" + 0.008*"fan" + 0.007*"original" + 0.006*"lp" + '
  '0.006*"later" + 0.006*"blue" + 0.006*"disc"'),
 (1,
  '0.011*"hook" + 0.010*"ep" + 0.010*"house" + 0.009*"dj" + 0.008*"funk" + '
  '0.008*"gucci" + 0.007*"verse" + 0.007*"rapper" + 0.007*"rap" + '
  '0.007*"sample"'),
 (2,
  '0.007*"ambient" + 0.007*"composition" + 0.006*"jazz" + 0.006*"techno" + '
  '0.006*"loop" + 0.005*"house" + 0.005*"instrument" + 0.005*"duo" + '
  '0.005*"percussion" + 0.005*"quality"'),
 (3,
  '0.045*"composer" + 0.031*"glass" + 0.029*"performance" + 0.028*"ensemble" + '
  '0.022*"jackson" + 0.022*"concert" + 0.020*"jazz" + 0.018*"expression" + '
  '0.017*"american" + 0.017*"opera"'),
 (4,
  '0.006*"punk" + 0.005*"metal" + 0.005*"friend" + 0.005*"young" + '
  '0.004*"death" + 0.004*"girl" + 0.003*"white" + 0.003*"riff" + 0.003*"die" + '
  '0.003*"maybe"'),
 (5,
  '0.105*"rap" + 0.048*"black" + 0.045*"rapper" + 0.015*"nigg

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.04493667267544

Coherence Score:  0.3866488739745399


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.096227 -0.055167       1        1  24.645918
2     -0.113995 -0.106396       2        1  15.514739
6     -0.024776 -0.037295       3        1  14.832294
0     -0.071358 -0.096641       4        1  13.096687
9     -0.046606 -0.111961       5        1  11.221361
7      0.094035 -0.135730       6        1   8.843572
1      0.143542 -0.072368       7        1   7.890247
5      0.238300  0.273603       8        1   2.256482
3     -0.291737  0.200764       9        1   1.238716
8     -0.023631  0.141191      10        1   0.459991, topic_info=     Category        Freq          Term       Total  loglift  logprob
82    Default  638.000000           rap  638.000000  30.0000  30.0000
1036  Default  486.000000         black  486.000000  29.0000  29.0000
83    Default  342.000000        rapper  342.000000  28.0000  28.0000
375   Default  333.000000   performance  333.000000  27.0000  27.0000
677   Default  323.000000          jazz  323.000000  26.0000  26.0000
855   Default  215.000000      composer  215.000000  25.0000  25.0000
237   Default  318.000000     recording  318.000000  24.0000  24.0000
1098  Default  414.000000         young  414.000000  23.0000  23.0000
1387  Default  260.000000   composition  260.000000  22.0000  22.0000
614   Default  262.000000      american  262.000000  21.0000  21.0000
1097  Default  272.000000         woman  272.000000  20.0000  20.0000
992   Default  115.000000        prince  115.000000  19.0000  19.0000
344   Default  224.000000          funk  224.000000  18.0000  18.0000
806   Default  255.000000      original  255.000000  17.0000  17.0000
1065  Default  333.000000         later  333.000000  16.0000  16.0000
511   Default  351.000000         house  351.000000  15.0000  15.0000
18    Default   99.000000         brown   99.000000  14.0000  14.0000
876   Default  254.000000        future  254.000000  13.0000  13.0000
353   Default  288.000000          hook  288.000000  12.0000  12.0000
102   Default  170.000000        street  170.000000  11.0000  11.0000
871   Default  247.000000          film  247.000000  10.0000  10.0000
2344  Default  248.000000       session  248.000000   9.0000   9.0000
1894  Default  249.000000         verse  249.000000   8.0000   8.0000
1157  Default  163.000000        player  163.000000   7.0000   7.0000
1812  Default  113.000000      ensemble  113.000000   6.0000   6.0000
2722  Default  132.000000     classical  132.000000   5.0000   5.0000
1137  Default  135.000000      language  135.000000   4.0000   4.0000
243   Default  285.000000        sample  285.000000   3.0000   3.0000
436   Default  252.000000       ambient  252.000000   2.0000   2.0000
484   Default  351.000000            ep  351.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
2239  Topic10   30.495604           ray   31.310030   5.3554  -3.5028
3066  Topic10   20.026531   piano_chord   20.841187   5.3418  -3.9233
3259  Topic10   17.972822  thematically   18.787386   5.3374  -4.0315
3064  Topic10   16.517899          meld   17.332529   5.3336  -4.1159
1539  Topic10   16.043650       fragile   16.858423   5.3322  -4.1451
3067  Topic10   15.741274          rein   16.555927   5.3313  -4.1641
1347  Topic10   14.979002       mindset   15.793622   5.3288  -4.2137
812   Topic10   14.808748   prominently   15.623374   5.3282  -4.2251
2203  Topic10   14.453450         ensue   15.267889   5.3269  -4.2494
3403  Topic10   14.432292   alternately   15.246771   5.3268  -4.2509
3411  Topic10   14.318063         sorry   15.132580   5.3264  -4.2588
2527  Topic10   13.555031           awe   14.369554   5.3234  -4.3136
3252  Topic10   10.795531     dichotomy   11.609983   5.3090  -4.5412
2622  Topic10   69.597359     christmas   77.150841   5.2787  -2.6776
3496  Topic10    6.515287         bored    7.329696   5.2639  -5.0462
3261  Topic10   20.